# Get Price and Volume Data then Calculate Simple Factors

This script is used to get price and volume data for stocks in our HSI universe generated from another notebook named Scrape HSI Constituents. 

Then, we will generate factor dataframe and return dataframe for further testing.

In this case, we will use Yahoo as pricing source and we need two items: Adj Close and Volume.

Adj Close is close price adjusted for split/spinoff and dividend, which is good to calculate total return and some momentum factors.

Volume is just trading volume. We will also calculate some factors from Volume.

Below is the list of factors that we will get:

{'126D Daily Return Skewness',
 '21D Daily Return High',
 '21D Daily Return Low',
 '21D Daily Return Skewness',
 '21D Daily Return Volatility',
 '21D Price EMA To 63D Price EMA',
 '21D Total Return',
 '252D Daily Return Skewness',
 '252D Total Return',
 '252D-21D Total Return',
 '50D Price EMA To 200D Price EMA',
 '63D Daily Return Skewness',
 '63D Daily Return Volatility',
 'Price To 252D Daily Price High',
 'Price To 252D Daily Price Low'}
 
Finally, we will archive two dataframes: hsiFactorDf and hsiReturnDf

In [1]:
import pandas as pd
import pandas_datareader.data as web
from dateutil.relativedelta import relativedelta

# Universe

In [2]:
hsiUniverseDf = pd.read_csv('hsiUniverseDf.csv', low_memory=False)
hsiUniverseDf['Update Date'] = hsiUniverseDf['Update Date'].astype('datetime64[ns]')

In [3]:
hsiUniverseDf.head()

,Stock Code,Update Date
0,0001.HK,2019-09-30
1,0002.HK,2019-09-30
2,0003.HK,2019-09-30
3,0005.HK,2019-09-30
4,0006.HK,2019-09-30


# Get Price and Volume Data from Yahoo

Unfortunately, some historical tickers' data are missing in Yahoo source. Luckily, it's not that much. I won't troubleshoot it in this demo script file. Probably you could change to other sources like quandl to tackle with missing data issue.

In [4]:
def getDataFromYahoo(universeDf):
    tickerList = list(set(universeDf['Stock Code']))
    startDate = universeDf['Update Date'].min() - relativedelta(years=2)
    endDate = universeDf['Update Date'].max()
    rawDataDfList = []
    for ticker in tickerList:
        try:
            thisDf = web.DataReader(ticker, 'yahoo', startDate, endDate)
            thisDf = thisDf.stack().reset_index()
            thisDf['Stock Code'] = ticker
            thisDf = thisDf.rename(columns={'level_1': 'Item', 'Date': 'Update Date', 0: 'Item Value'})
            thisDf = thisDf[thisDf['Item'].isin(['Adj Close', 'Volume'])]
            rawDataDfList.append(thisDf)
            print('Downloaded Data for ' + ticker)
        except:
            print('Failed Downloading for ' + ticker)
            pass
    rawDataDf = pd.concat(rawDataDfList)
    return rawDataDf

In [5]:
rawDataDf = getDataFromYahoo(universeDf=hsiUniverseDf)

Downloaded Data for 2388.HK
Downloaded Data for 1898.HK
Downloaded Data for 0066.HK
Downloaded Data for 0001.HK
Downloaded Data for 0823.HK
Downloaded Data for 1113.HK
Downloaded Data for 0011.HK
Downloaded Data for 0012.HK
Downloaded Data for 1093.HK
Downloaded Data for 0386.HK
Downloaded Data for 1398.HK
Downloaded Data for 1199.HK
Downloaded Data for 0003.HK
Downloaded Data for 3988.HK
Downloaded Data for 0688.HK
Downloaded Data for 0017.HK
Downloaded Data for 1044.HK
Downloaded Data for 2319.HK
Downloaded Data for 0004.HK
Downloaded Data for 2018.HK
Downloaded Data for 0669.HK
Downloaded Data for 0883.HK
Downloaded Data for 2313.HK
Failed Downloading for 0013.HK
Downloaded Data for 1177.HK
Downloaded Data for 0992.HK
Downloaded Data for 2318.HK
Downloaded Data for 0083.HK
Downloaded Data for 1928.HK
Downloaded Data for 1109.HK
Downloaded Data for 3328.HK
Downloaded Data for 2600.HK
Downloaded Data for 1038.HK
Downloaded Data for 0267.HK
Downloaded Data for 0019.HK
Downloaded Data f

In [6]:
rawDataDf.head()

,Update Date,Item,Item Value,Stock Code
4,2007-10-02,Volume,4.336922e+07,2388.HK
5,2007-10-02,Adj Close,1.209313e+01,2388.HK
10,2007-10-03,Volume,6.268460e+07,2388.HK
11,2007-10-03,Adj Close,1.179759e+01,2388.HK
16,2007-10-04,Volume,3.277097e+07,2388.HK


# Calculate Simple Factors

In [29]:
class calcFactorDf():
    
    def __init__(self, universeDf, rawDataDf):
        self.universeDf = universeDf
        self.rawDataDf = rawDataDf
        
    def run(self):
        inputUniverseDf = self.universeDf
        inputRawDataDf = self.rawDataDf
        
        def getPivotDf(rawDataDf, item):
            pivotDataDf = pd.pivot_table(rawDataDf[rawDataDf['Item'] == item], 
                                         index='Update Date', columns='Stock Code', values='Item Value')
            return pivotDataDf
        
        def calcMomentumDf(universeDf, adjCloseDf, dayLength):
            adjCloseChgDf = adjCloseDf.pct_change(periods=dayLength).stack().reset_index().rename(columns={0: 'Factor Value'})
            momentumFactorDf = pd.merge(universeDf, adjCloseChgDf, on=['Stock Code', 'Update Date'], how='left')
            momentumFactorDf['Factor Name'] = str(dayLength) + 'D Total Return'
            momentumFactorDf['Factor Value'] *= 100 
            del adjCloseChgDf
            return momentumFactorDf
        
        def calcMomentumDiffDf(universeDf, longTermMomentumDf, shortTermMomentumDf):
            longTermLength = longTermMomentumDf.iloc[0, -1].split(' ')[0]
            shortTermLength = shortTermMomentumDf.iloc[0, -1].split(' ')[0]
            thisMomentumName = longTermLength + '-' + shortTermLength + ' Total Return'
            mergedColumnsList = ['Stock Code', 'Update Date', 'Factor Value']
            thisFactorDf = pd.merge(universeDf, 
                                    longTermMomentumDf[mergedColumnsList].rename(columns={'Factor Value': 'LT'}),
                                    on=['Stock Code', 'Update Date'], how='left')
            thisFactorDf = pd.merge(thisFactorDf, 
                                    shortTermMomentumDf[mergedColumnsList].rename(columns={'Factor Value': 'ST'}),
                                    on=['Stock Code', 'Update Date'], how='left')
            thisFactorDf['Factor Value'] = thisFactorDf['LT'] - thisFactorDf['ST']
            thisFactorDf = thisFactorDf.drop(columns=['LT', 'ST'])
            thisFactorDf['Factor Name'] = thisMomentumName
            del longTermLength, shortTermLength, thisMomentumName, mergedColumnsList
            return thisFactorDf
        
        def calcNonRangeItems(universeDf, adjCloseDf, dayLength, itemType, category):
            itemDfList = []
            functionString = ('std' if itemType == 'Volatility' else 
                              'max' if itemType == 'High' else 
                              'min' if itemType == 'Low' else 
                              'skew' if itemType == 'Skewness' else None)
            updateDateList = list(universeDf['Update Date'].drop_duplicates())
            if category == 'Return':
                returnDf = adjCloseDf.pct_change()
            elif category == 'Price':
                returnDf = adjCloseDf.copy()
            else:
                raise Exception('category only accepts Return or Price')
                
            for date in updateDateList:
                thisReturnDf = returnDf[(returnDf.index > date - relativedelta(days=dayLength)) & 
                                        (returnDf.index <= date)].copy()
                thisReturnDf = thisReturnDf.dropna(how='all', axis=1)
                thisItemDf = pd.DataFrame(getattr(thisReturnDf, functionString)()).reset_index().rename(columns={0: 'Factor Value'})
                thisItemDf['Update Date'] = date
                itemDfList.append(thisItemDf)
                del thisReturnDf, thisItemDf

            itemDf = pd.concat(itemDfList)
            itemDf = pd.merge(universeDf, itemDf, on=['Stock Code', 'Update Date'], how='left')
            
            itemDf['Factor Name'] = str(dayLength) + 'D Daily ' + category + ' ' + itemType

            if itemType != 'Skewness' and category == 'Return':
                itemDf['Factor Value'] *= 100

            del itemDfList, functionString, updateDateList, returnDf
            return itemDf
        
        def calcCurrentToHighLowPrice(universeDf, adjCloseDf, dayLength, itemType, category):
            highLowPriceDf = calcNonRangeItems(universeDf=universeDf, adjCloseDf=adjCloseDf, 
                                               dayLength=dayLength, itemType=itemType, category=category)
            stackCloseDf = adjCloseDf.stack().reset_index().rename(columns={0: 'Adj Close'})
            priceToHighLowDf = pd.merge(highLowPriceDf, stackCloseDf, on=['Stock Code', 'Update Date'], how='left')
            priceToHighLowDf['Price To High/Low'] = priceToHighLowDf['Adj Close'] / priceToHighLowDf['Factor Value']
            thisFactorName = 'Price To ' + priceToHighLowDf.iloc[0, priceToHighLowDf.columns.get_loc('Factor Name')]
            priceToHighLowDf = priceToHighLowDf.drop(columns=['Factor Value', 'Adj Close', 'Factor Name'])
            priceToHighLowDf = priceToHighLowDf.rename(columns={'Price To High/Low': 'Factor Value'})
            priceToHighLowDf = pd.merge(universeDf, priceToHighLowDf, on=['Stock Code', 'Update Date'], how='left')
            priceToHighLowDf['Factor Name'] = thisFactorName
            
            del highLowPriceDf, stackCloseDf, thisFactorName
            return priceToHighLowDf
        
        def calcMovingAverage(universeDf, dataDf, dataType, dayLength):
            thisEMADf = dataDf.ewm(span=dayLength, min_periods=int(dayLength/2), adjust=True, ignore_na=True).mean()
            thisEMADf = thisEMADf.stack().reset_index().rename(columns={0: str(dayLength) + 'D ' + dataType + ' EMA'})
            thisEMADf = pd.merge(universeDf, thisEMADf, on=['Stock Code', 'Update Date'], how='left')
            return thisEMADf

        def calcShortEMAToLongEMA(universeDf, shortDayLength, longDayLength, dataDf, dataType):
            shortEMADf = calcMovingAverage(universeDf=universeDf, dataDf=dataDf, dataType=dataType, dayLength=shortDayLength)
            longEMADf = calcMovingAverage(universeDf=universeDf, dataDf=dataDf, dataType=dataType, dayLength=longDayLength)

            shortName = [column for column in shortEMADf.columns if column not in ['Stock Code', 'Update Date']][0]
            longName = [column for column in longEMADf.columns if column not in ['Stock Code', 'Update Date']][0]

            shortEMAToLongEMADf = pd.merge(shortEMADf, longEMADf, on=['Stock Code', 'Update Date'], how='left')

            shortEMAToLongEMADf['Factor Value'] = shortEMAToLongEMADf[shortName] / shortEMAToLongEMADf[longName]
            shortEMAToLongEMADf['Factor Name'] = shortName + ' To ' + longName

            shortEMAToLongEMADf = shortEMAToLongEMADf.drop(columns=[shortName, longName])
            del shortEMADf, longEMADf, shortName, longName
            return shortEMAToLongEMADf
            
        adjCloseDf = getPivotDf(rawDataDf=inputRawDataDf, item='Adj Close')
        volumeDf = getPivotDf(rawDataDf=inputRawDataDf, item='Volume')
        
        totalReturn252Df = calcMomentumDf(universeDf=inputUniverseDf, adjCloseDf=adjCloseDf, dayLength=252)
        totalReturn21Df = calcMomentumDf(universeDf=inputUniverseDf, adjCloseDf=adjCloseDf, dayLength=21)
        totalReturn252_21Df = calcMomentumDiffDf(universeDf=inputUniverseDf, 
                                                 longTermMomentumDf=totalReturn252Df, 
                                                 shortTermMomentumDf=totalReturn21Df)
        
        totalReturnSkew21Df = calcNonRangeItems(universeDf=inputUniverseDf, adjCloseDf=adjCloseDf, 
                                                dayLength=21, itemType='Skewness', category='Return')
        totalReturnSkew63Df = calcNonRangeItems(universeDf=inputUniverseDf, adjCloseDf=adjCloseDf, 
                                                dayLength=63, itemType='Skewness', category='Return')
        totalReturnSkew126Df = calcNonRangeItems(universeDf=inputUniverseDf, adjCloseDf=adjCloseDf, 
                                                 dayLength=126, itemType='Skewness', category='Return')
        totalReturnSkew252Df = calcNonRangeItems(universeDf=inputUniverseDf, adjCloseDf=adjCloseDf, 
                                                 dayLength=252, itemType='Skewness', category='Return')
        
        totalReturnHigh21Df = calcNonRangeItems(universeDf=inputUniverseDf, adjCloseDf=adjCloseDf, 
                                                dayLength=21, itemType='High', category='Return')
        totalReturnLow21Df = calcNonRangeItems(universeDf=inputUniverseDf, adjCloseDf=adjCloseDf, 
                                               dayLength=21, itemType='Low', category='Return')
        
        totalReturnVol21Df = calcNonRangeItems(universeDf=inputUniverseDf, adjCloseDf=adjCloseDf,
                                               dayLength=21, itemType='Volatility', category='Return')
        totalReturnVol63Df = calcNonRangeItems(universeDf=inputUniverseDf, adjCloseDf=adjCloseDf, 
                                               dayLength=63, itemType='Volatility', category='Return')
        
        priceToHigh252Df = calcCurrentToHighLowPrice(universeDf=inputUniverseDf, adjCloseDf=adjCloseDf, 
                                                   dayLength=252, itemType='High', category='Price')
        
        priceToLow252Df = calcCurrentToHighLowPrice(universeDf=inputUniverseDf, adjCloseDf=adjCloseDf, 
                                                   dayLength=252, itemType='Low', category='Price')
        
        ema50ToEMA200Df = calcShortEMAToLongEMA(universeDf=hsiUniverseDf, shortDayLength=50, longDayLength=200, 
                                                dataDf=adjCloseDf, dataType='Price')
        
        volume21ToVolume63Df = calcShortEMAToLongEMA(universeDf=hsiUniverseDf, shortDayLength=21, longDayLength=63, 
                                                dataDf=volumeDf, dataType='Price')
        
        factorDf = pd.concat([totalReturn252Df, totalReturn21Df, totalReturn252_21Df, 
                              totalReturnSkew21Df, totalReturnSkew63Df, totalReturnSkew126Df, totalReturnSkew252Df,
                              totalReturnHigh21Df, totalReturnLow21Df, 
                              totalReturnVol21Df, totalReturnVol63Df, 
                              priceToHigh252Df, priceToLow252Df, 
                              ema50ToEMA200Df, volume21ToVolume63Df])
        
        return factorDf

In [30]:
factorDf = calcFactorDf(universeDf=hsiUniverseDf, rawDataDf=rawDataDf).run()

In [31]:
factorDf[factorDf['Stock Code'] == '0001.HK'].sort_values('Update Date')

,Stock Code,Update Date,Factor Value,Factor Name
5869,0001.HK,2009-09-30,0.921763,21D Price EMA To 63D Price EMA
5869,0001.HK,2009-09-30,9.661233,252D Total Return
5869,0001.HK,2009-09-30,1.827043,Price To 252D Daily Price Low
5869,0001.HK,2009-09-30,8.002268,21D Total Return
5869,0001.HK,2009-09-30,0.963405,Price To 252D Daily Price High
...,...,...,...,...
0,0001.HK,2019-09-30,0.938983,21D Daily Return Volatility
0,0001.HK,2019-09-30,0.943847,50D Price EMA To 200D Price EMA
0,0001.HK,2019-09-30,0.426571,252D Daily Return Skewness
0,0001.HK,2019-09-30,1.076695,Price To 252D Daily Price Low


In [56]:
factorDf.to_csv('hsiFactorDf.csv', index=False)

# Generate Daily Return DataFrame

In [53]:
def getReturnDf(rawDataDf):
    returnDf = rawDataDf[rawDataDf['Item'] == 'Adj Close'].copy()
    returnDf = pd.pivot_table(returnDf, index='Update Date', columns='Stock Code', values='Item Value')
    returnDf = returnDf.pct_change().stack().reset_index().rename(columns={0: 'Return'})
    return returnDf

In [54]:
returnDf = getReturnDf(rawDataDf=rawDataDf)

In [55]:
returnDf

,Update Date,Stock Code,Return
0,2007-10-03,0001.HK,-0.050000
1,2007-10-03,0002.HK,-0.017723
2,2007-10-03,0003.HK,0.088397
3,2007-10-03,0004.HK,-0.042672
4,2007-10-03,0005.HK,0.024983
...,...,...,...
182053,2019-09-30,2388.HK,0.005671
182054,2019-09-30,2600.HK,0.000000
182055,2019-09-30,2628.HK,0.002207
182056,2019-09-30,3328.HK,0.007874


In [57]:
returnDf.to_csv('hsiReturnDf.csv', index=False)